# Oracle Consensus Algorithms

In [38]:
import numpy as np
from typing import List, Tuple

## Sample Data

In [39]:
N_oracles = 20
failing_percentage = 0.2
N_failing_oracles = round(N_oracles * failing_percentage)


In [40]:
def normalize(x) :
    return (np.arctan(x) / np.pi) + 0.5

def denormalize(y) :
    return np.tan(np.pi * (y - 0.5))

def generate_oracles(N_oracles, N_failing_oracles, e=0, sigma=1):
    oracles = [
        (normalize(np.random.normal(e, sigma)), True)
        if i >= N_failing_oracles 
        else (np.random.uniform(0, 1), False)
        for i in range(N_oracles)
    ]

    np.random.shuffle(oracles)

    disjoint = tuple_listes = tuple(map(list, zip(*oracles)))

    return disjoint

true_essence = 0
true_sigma = 1


oracles, true_oracles = generate_oracles(N_oracles, N_failing_oracles, true_essence, true_sigma)

## Simple Algorithm

In [41]:
def essence_estimate(oracles : List[Tuple[float, bool]]):
    '''
        By default, average
    '''

    weight = 0
    result = 0

    for value, activated in oracles:
        if activated :
            weight += 1
            result += value
    
    return result / weight


def one_oracle_score(oracle_value, essence_estimate):
    '''
        Let's say the oracle score is defined by the squared euclidean distance to the essence estimate
    '''
    return np.linalg.norm(oracle_value - essence_estimate) ** 2


def compute_scores(oracles : List[Tuple[float, bool]], only_active : bool = False) :
    essence = essence_estimate(oracles)
    return [
        one_oracle_score(value, essence)
        if activated or not only_active else None
        for value, activated in oracles
    ]

def argmin(T):
    min_value = None 
    min_idx = None
    for idx, value in enumerate(T):
        if value is None : continue

        if min_value is None :
            min_value = value
            min_idx = idx
            continue

        if value < min_value :
            min_value = value
            min_idx = idx

    return min_idx

def find_the_worst_oracle(oracles : List[Tuple[float, bool]]) :
    return argmin(compute_scores(oracles, True))

def remove_worst_oracles(oracles : List[float], N_failing_oracles : int) :
    result = [(oracle, True) for oracle in oracles]

    for _ in range(N_failing_oracles):
        worst = find_the_worst_oracle(result) 
        result[worst] = (result[worst][0], False)

    return result, compute_scores(result)

result, scores = remove_worst_oracles(oracles, N_failing_oracles)

In [42]:
for i in range(len(result)) :
    expected = true_oracles[i]
    found = result[i][1]
    s = "[ ]" if expected == found else "[X]"

    print(f"- {s}\t Expected : {expected} \t Found : {found} \t Score : {scores[i]}")

- [ ]	 Expected : False 	 Found : False 	 Score : 0.005539464763822209
- [X]	 Expected : True 	 Found : False 	 Score : 0.002421227690621136
- [ ]	 Expected : True 	 Found : True 	 Score : 0.16815244873041238
- [ ]	 Expected : True 	 Found : True 	 Score : 0.12661957836652465
- [ ]	 Expected : True 	 Found : True 	 Score : 0.008132612158463028
- [ ]	 Expected : False 	 Found : False 	 Score : 0.00041271397805076374
- [ ]	 Expected : True 	 Found : True 	 Score : 0.12583723867331897
- [X]	 Expected : False 	 Found : True 	 Score : 0.012433813633480065
- [ ]	 Expected : True 	 Found : True 	 Score : 0.03807057291944793
- [ ]	 Expected : True 	 Found : True 	 Score : 0.041201764407771
- [X]	 Expected : True 	 Found : False 	 Score : 0.005827147347858098
- [ ]	 Expected : True 	 Found : True 	 Score : 0.03214902479206026
- [X]	 Expected : False 	 Found : True 	 Score : 0.07876921954797243
- [ ]	 Expected : True 	 Found : True 	 Score : 0.07313594362261162
- [ ]	 Expected : True 	 Found : T

In [45]:
normalized_error = np.abs(essence_estimate(result) - true_essence) 
denormalized_error = np.abs(denormalize(essence_estimate(result)) - denormalize(true_essence))

print(f"{normalized_error} \t {denormalized_error}")

0.5766983183486516 	 1.633123935319537e+16


The first method is clearly not effective.
Possibles issues :
- Current essence estimate would work for a gaussian noise but not for the normalized gaussian. 